In [1]:
#run this cell when online
!pip install xlrd

You are using pip version 10.0.1, however version 19.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## install dependencies and set data paths

In [1]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

In [2]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [3]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/bch'

In [4]:
#loading in the entire spreadsheet as a dataframe
clin = pd.read_excel(data_path + '/BCH ED visits FY1819_with additions.xlsx')

In [5]:
len(clin)

136993

In [6]:
clin.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading'],
      dtype='object')

In [7]:
clin.dropna(subset = ['SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory'], inplace = True)
clin.shape

(102543, 27)

## The next few cells are for getting a feel of the data in the different columns and how they might be useful

In [9]:
discharge_count = Counter(clin['DischargeDisposition']); discharge_count

Counter({17: 70703,
         62: 762,
         7: 11199,
         6: 374,
         16: 493,
         40: 226,
         8: 242,
         72: 113,
         63: 250,
         30: 346,
         64: 171,
         12: 242,
         71: 3,
         14: 3,
         61: 12,
         90: 10,
         13: 3,
         9: 2})

In [10]:
discharge_count2 = Counter(clin['DischargeDispositionDesc']); discharge_count2

Counter({'Discharge to private home, condo, apt without support service/referral': 70703,
         'Left at his/her own risk post-initial treatment': 762,
         'Admit to reporting facility as inpatient to another unit from amb care': 11199,
         'Admit to reporting facility as inpatient to SCU or OR from amb care': 374,
         'Discharge to private home, condo, apt with support service/referral': 493,
         'Transfer to Group/supportive living': 226,
         'Transfer to another acute care facility directly from amb care': 242,
         'Died in Facility': 113,
         'Left After Triage': 250,
         'Transfer to Residential care': 346,
         'Left After Initial Assessment': 171,
         'Intra-facility transfer to day surgery': 242,
         'Dead on arrival': 3,
         'Intra-facility transfer to clinic': 3,
         'Left at his/her own risk following registration': 12,
         'Transfer to correctional facility': 10,
         'Intra-facility transfer to ED'

In [11]:
comp_count = Counter(clin['PatientService']); comp_count

Counter({nan: 73660,
         'Cardiology': 1550,
         'ALC General Medicine': 201,
         'General Medicine': 4512,
         'INTENSIVE CARE UNIT': 253,
         'Respirology': 852,
         'General Surgery': 650,
         'Paediatrics': 882,
         'Mental Health': 1198,
         'Orthopaedics': 430,
         'ALC Neurology': 64,
         'Genitourinary': 182,
         'Neurology': 456,
         'Palliative': 65,
         'ALC Respirology': 28,
         'Oncology': 38,
         'ALC Orthopaedics': 23,
         'ALC Cardiology': 38,
         'AD PALLIATIVE': 45,
         'AD General Medicine': 2,
         'Obstetrics': 17,
         'AD Respirology': 1,
         'AD Oncology': 1,
         'ALC General Surgery': 2,
         'Neonatal Retro Transfers': 1,
         'ALC Oncology': 1,
         'AD Neurology': 1,
         'Newborn': 1})

## refactoring some of the cells so they are clean and useful

In [8]:
o2sat = [item if item[-1] != '%' else item[:-1] for item in clin['O2Saturation_LastEDReading'].astype('str')]
o2sat = [np.nan if item == 'nan' else float(item) for item in o2sat] 
clin['o2sat'] = o2sat

In [9]:
pulse = clin['Pulse_LastEDReading']
pulse = [re.sub("[^0-9]", "", str(item)) for item in pulse]
pulse = [np.nan if item == '' else float(item) for item in pulse] 
#set(pulse)
clin['pulse'] = pulse

In [10]:
temp = clin['Temperature_LastEDReading']
temp = [re.sub("[^0-9]", "", str(item)) for item in temp]
temp = [np.nan if item == '' else float(item) for item in temp] 
temp = [item/10 if item > 100 else item for item in temp]
temp = [np.nan if item > 40  else item for item in temp]
#set(temp)
clin['temp'] = temp

In [11]:
def bptrans(bp):
    if pd.isnull(bp) or len(bp.split('/')) !=2:
        return [np.nan, np.nan]
    res = []
    for x in bp.split('/'):
        try:
            float(x)
            res =  [float(x) for x in bp.split('/')]
        except: 
            res =  [np.nan, np.nan]
        return res


In [12]:
#this is to clean the text in the subjective notes column (a little bit)
def fixencode(s):
    if pd.isnull(s):
        s = "None"
    s = s[14:]
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace('htn', 'hypertension')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [13]:
clin['CleanSubjectiveNotes'] = clin.SubjectiveNotes.map(fixencode)

In [15]:
for item in clin['CleanSubjectiveNotes'].iloc[10000:10010]:
    print (item)

patient states feeling depressed for few months,last tuesday patient tried to harm herself by taking 10 extra strength tylenol. denies any intentions to harm herself now or any intention to harm others,
patient was playing around at school, was pushed by fellow student, patient felt a twisting sensation on her left knee and fell to floor. hurts to ambulate on, good csm
as per mother, patient was in the school bus yesterday and it was hit form behind. since then patient has been exp nausea, headaches and stomach aches. no vomits or diarrhea. no urinary or bowel symptoms. no neuro deficits noted. no resp distres.
sst fell on ladder this am,landed to right foot, complains of pain with swelling, sts no head injury, concerned abt his foot, sts took tylenol #3 2 tabs, abt 10 am.sts unable to wieght bear.
as per patient with pain to lateral left back (describes as stabbing pain) radiating across mid back x1 month. seen at wic and bw done and patient sent to emergency department with elevated 

In [16]:
clin['BP'] = clin.BloodPressure_LastEDReading.map(bptrans)

clin[['systolic', 'diastolic']] = pd.DataFrame(clin.BP.tolist(), index= clin.index)
clin["Gender"] = clin.GenderDesc.map(lambda x: 1 if x == "Male" else 0)

## split the screening questions
- result is a df called: screening_df

In [17]:
qlist = ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
         'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
        'Do you have a new Rash?',
        'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
         'Have you travelled outside of Canada/USA in the last 3 weeks?',
         'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
         'Have you received Health Care in another country in the last 2 years?',
        'Do you have a new/worse cough or shortness of breath?',
         'If so, select all countries that apply',
        'If so, select all infectious diseases that apply']

In [18]:
#making a dict so that the indect of an item on the list, matches the question later
q_dict = {i:q for i,q in enumerate(qlist)}

In [19]:
q_dict

{0: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 1: 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 2: 'Do you have a new Rash?',
 3: 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 4: 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 5: 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 6: 'Have you received Health Care in another country in the last 2 years?',
 7: 'Do you have a new/worse cough or shortness of breath?',
 8: 'If so, select all countries that apply',
 9: 'If so, select all infectious diseases that apply'}

In [20]:
#this is going to be a list made of up all the entries in the infection control screening column
#casting the results as string to manipulate later
qans = list(clin['InfectionControlScreening'].str[14:].astype('str'))

In [21]:
#this is going to give me a way to split the questions by finding where they occur in the field
split_points = []
for item in qans:
    temp_points = []
    for word in qlist:
        temp_points.append(item.find(word))
    split_points.append(temp_points)
    

In [22]:
#this is going to give me list of places to split each entry (using only the questions that are actually present)
real_points = []
for item in split_points:
    real_points.append([idx for idx in sorted(item) if idx > 0])

In [23]:
split_points[10000:10010]

[[241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1]]

In [24]:
#anything from the list above which would have been empty in the original data
#gives a list of -1's in split_points and nothing in real_points
real_points[10000:10010]

[[70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400]]

In [25]:
print(qans[1000])
print()
print (split_points[1000])
print ()
print(real_points[1000])

Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Do you have a new/worse cough or shortness of breath? Y<LT>LF>Are you feeling feverish or have had shakes or chills in the last 24 hours? Y<LT>LF>Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>Do you have a new Rash? N

[241, -1, 400, 325, 0, 70, -1, 179, -1, -1]

[70, 179, 241, 325, 400]


In [26]:
#this should mutate qans so each item is now a list of q and a's
for idx in range(len(qans)):
    qans[idx] = [qans[idx][i : j] for i, j in zip([0] + real_points[idx], real_points[idx] + [None])]
    

In [27]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? Y<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? Y<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [28]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? N<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [29]:
#this will mutate each item of each list in qans and drop the <LT>LF> if present
for q in qans:
    for idx in range(len(q)):
        if q[idx].find('<LT>LF>') != -1:
            q[idx] = q[idx][:-7]
        

In [30]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? Y',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? Y',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [31]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [32]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? Y',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? Y',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [33]:
seg_list = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = []
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                #print ('question:',item[:len(qq)])
                #print ('answer:',item[len(qq)+1:])
                res.append([item[:len(qq)],item[len(qq)+1:] ])
    seg_list.append(res)
    #print ('\nresult:',res, '\n\n')

In [34]:
len(seg_list)

102543

In [35]:
seg_list2 = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = {}
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                question = item[:len(qq)]
                answer = item[len(qq)+1:]
                res[question] = answer
    seg_list2.append(res)

In [36]:
final_list = []
for q in range(len(split_points)):
    temp = []
    for i in range(len(split_points[q])):
        if split_points[q][i] == -1:
            #print('nan')
            temp.append('nan')
        else:
            #print (brief_list2[q][q_dict[i]])  # q_dict[i], i, brief_splits[q][i],
            temp.append(seg_list2[q][q_dict[i]])
    final_list.append(temp)
    #print ('\n next item \n')
            

In [37]:
len(final_list)

102543

In [38]:
final_list[100:110]

[['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'nan', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan']]

In [39]:
final_list[1000:1020]

[['Y', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'Y', 'N', 'nan', 'Y', 'IND', 'nan'],
 ['N', 'nan', 'N', 'nan', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['Y', 'nan', 'N', 'N', 'Y', 'N', 'nan', 'N', 'IND', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N

In [40]:
screening_df = pd.DataFrame(final_list, columns = qlist)
screening_df.head()

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
0,Y,nan,N,N,N,N,nan,N,nan,nan
1,Y,nan,N,Y,N,N,nan,N,nan,nan
2,N,nan,N,N,N,N,nan,N,nan,nan
3,N,nan,N,N,N,N,nan,Y,nan,nan
4,N,nan,N,N,N,N,nan,N,nan,nan


In [41]:
screening_df.shape

(102543, 10)

In [42]:
screening_df.iloc[80000:80010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
80000,N,nan,N,N,N,N,nan,N,nan,nan
80001,N,nan,nan,nan,N,nan,nan,N,nan,nan
80002,Y,nan,N,N,N,N,nan,Y,nan,nan
80003,Y,nan,nan,N,N,N,nan,N,nan,nan
80004,N,nan,N,N,N,N,nan,N,nan,nan
80005,N,nan,N,N,N,N,nan,N,nan,nan
80006,N,nan,N,N,N,N,nan,N,nan,nan
80007,N,nan,N,N,N,N,nan,N,nan,nan
80008,Y,nan,N,N,Y,N,nan,Y,USA,nan
80009,N,nan,N,N,N,N,nan,N,nan,nan


In [43]:
screening_df.replace('nan',np.NaN, inplace = True)

In [44]:
screening_df.iloc[80000:80010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
80000,N,NaN,N,N,N,N,NaN,N,NaN,NaN
80001,N,NaN,NaN,NaN,N,NaN,NaN,N,NaN,NaN
80002,Y,NaN,N,N,N,N,NaN,Y,NaN,NaN
80003,Y,NaN,NaN,N,N,N,NaN,N,NaN,NaN
80004,N,NaN,N,N,N,N,NaN,N,NaN,NaN
80005,N,NaN,N,N,N,N,NaN,N,NaN,NaN
80006,N,NaN,N,N,N,N,NaN,N,NaN,NaN
80007,N,NaN,N,N,N,N,NaN,N,NaN,NaN
80008,Y,NaN,N,N,Y,N,NaN,Y,USA,NaN
80009,N,NaN,N,N,N,N,NaN,N,NaN,NaN


## making the new target cells

In [45]:
admit_count= Counter(clin['AdmitLocation']); admit_count

Counter({nan: 92747,
         'N.SUR ORTH': 326,
         'N.MH CHADI': 113,
         'N.SUR SS': 448,
         'N.MH CHAD': 157,
         'N.MED CARD': 931,
         'N.MAU': 1544,
         'N.MED ONC': 591,
         'N.MED GER': 484,
         'N.CC ICU': 122,
         'N.SIMCU': 135,
         'N.SUR GEN': 714,
         'N.MED NEUR': 451,
         'N.MED CT': 97,
         'N.CC CCU': 292,
         'N.ER IN': 670,
         'N.MH INTEN': 156,
         'N.WC PAED': 860,
         'N.MH GER': 42,
         'N.MED RESP': 499,
         'N.MED SIM': 305,
         'N.MH GENC': 38,
         'N.MH GENB': 164,
         'N.MED SIX': 314,
         'N.MED DIAL': 29,
         'N.MED CPU': 152,
         'N.ERMH IN': 18,
         'N.WC POST': 88,
         'N.MAIN OR': 1,
         'N.CC NICU': 1,
         'N.MED FLEX': 4,
         'N.MED DSU': 49,
         'N.SUR DSU': 1})

In [46]:
clin['outcome'] = clin['AdmitLocation'].astype('str').map(
{'N.CC CCU': 'madmit',
 'N.CC ICU': 'ICU',
 'N.ER IN' : 'madmit',
 'N.ERMH IN': 'madmit',
 'N.MAU': 'madmit',
 'N.MED CARD': 'madmit',
 'N.MED CPU': 'madmit',
 'N.MED CT': 'madmit',
 'N.MED GER': 'madmit',
 'N.MED DIAL': 'madmit',
 'N.MED NEUR': 'madmit',
 'N.MED ONC': 'madmit',
 'N.MED RESP': 'madmit',
 'N.MED SIM': 'madmit',
 'N.MED SIX': 'madmit',
 'N.MH CHAD': 'madmit',
 'N.MH CHADI': 'madmit',
 'N.MH GENB': 'madmit',
 'N.MH GER': 'madmit',
 'N.MH INTEN': 'madmit',
 'N.SIMCU': 'ICU',
 'N.SUR GEN': 'sadmit',
 'N.SUR ORTH': 'sadmit',
 'N.SUR SS': 'sadmit',
 'N.WC PAED': 'madmit',
 'N.WC POST': 'madmit',
 'N.MH GENC': 'madmit',
 'N.CC NICU': 'ICU',
 'N.WC LD': 'sadmit',
  'N.WC LDN': 'sadmit',
  'N.MAIN OR': 'sadmit',
  'N.MED STTU': 'madmit',
  'N.MED FLEX': 'madmit',
    'N.MED DSU': 'madmit',
    'N.SUR DSU': 'sadmit',
 'nan': 'discharge'})

In [47]:
#first target columns is based on the admitting location
clin['target'] = clin['outcome'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [48]:
outcome_count = Counter(clin['outcome']); outcome_count

Counter({'discharge': 92747, 'sadmit': 1490, 'madmit': 8048, 'ICU': 258})

In [49]:
target_count = Counter(clin['target']); target_count

Counter({1: 92747, 3: 1490, 2: 8048, 4: 258})

In [50]:
clin['service'] = clin['PatientService'].astype('str').map(
    {'nan': 'discharge',
         'General Surgery': 'sadmit',
         'Respirology': 'madmit',
         'General Medicine': 'madmit',
         'INTENSIVE CARE UNIT': 'ICU',
         'Cardiology': 'madmit',
         'Oncology': 'madmit',
         'Palliative': 'madmit',
         'AD PALLIATIVE': 'madmit',
         'Genitourinary': 'sadmit',
         'Paediatrics': 'madmit',
         'Mental Health': 'madmit',
         'Orthopaedics': 'sadmit',
         'MH Child & Adolescent': 'madmit',
         'AD Oncology': 'madmit',
         'Neurology': 'madmit',
         'ALC General Medicine': 'madmit',
         'ALC General Surgery': 'sadmit',
         'Acute Care for Elders': 'madmit',
         'ALC Neurology': 'madmit',
         'ALC Cardiology': 'madmit',
         'ALC Respirology': 'madmit',
         'ALC Orthopaedics': 'madmit',
         'Neonatal Retro Transfers': 'madmit',
         'Newborn': 'madmit',
         'Obstetrics': 'madmit',
         'ALC Oncology': 'madmit',
         'AD General Medicine': 'madmit',
         'AD Acute Care for Elders': 'madmit',
         'AD Respirology': 'madmit',
         'ALC ACUTE CARE FOR EDLERS': 'madmit',
         'AD Cardiology': 'madmit',
         'AD Neurology': 'madmit',
         'ALC Palliative': 'madmit',
         'AD General Surgery': 'sadmit'})

In [51]:
service_count = Counter(clin['service'])
service_count

Counter({'discharge': 92747, 'sadmit': 1725, 'madmit': 7901, 'ICU': 170})

In [52]:
#second target is based on the admitting service
clin['target2'] = clin['service'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [53]:
target2_count = Counter(clin['target2']); target2_count

Counter({1: 92747, 3: 1725, 2: 7901, 4: 170})

In [54]:
#creating this for later use to map the numbers in the dataframe column to their original meaning
out_dict = {'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4}
out_dict = {v:k for k,v in out_dict.items()}
out_dict

{1: 'discharge', 2: 'madmit', 3: 'sadmit', 4: 'ICU'}

In [55]:
#discharge vs. not target
clin['discharge'] = clin['outcome'].map({'discharge': 'discharge', 'madmit': 'admit', 'sadmit': 'admit', 'ICU': 'admit'})

In [56]:
clin['target3'] = clin['outcome'].map({'discharge': 1, 'madmit': 0, 'sadmit': 0, 'ICU': 0})

In [57]:
Counter(clin['discharge']), Counter(clin['target3'])

(Counter({'discharge': 92747, 'admit': 9796}), Counter({1: 92747, 0: 9796}))

In [58]:
#a target for trying to tell apart the inpatient disposition, only in admitted patients
clin['dispo']= clin['outcome'].map({'discharge': np.nan, 'madmit': 'madmit', 'sadmit': 'sadmit', 'ICU': 'ICU'})

In [59]:
clin['target4'] = clin['dispo'].map({np.nan:np.nan, 'madmit': 1, 'sadmit': 2, 'ICU': 3})
#for some reason a can't make a counter out of this without it crashing so I've stopped trying

In [60]:
Counter(clin['dispo']), Counter(clin['target4'].dropna())

(Counter({nan: 92747, 'sadmit': 1490, 'madmit': 8048, 'ICU': 258}),
 Counter({2.0: 1490, 1.0: 8048, 3.0: 258}))

In [61]:
#gonna make a target that is ICU vs other
clin['ICUvsother'] = clin['PatientService'][clin['PatientService'] == 'INTENSIVE CARE UNIT']
#clin['PatientService']

In [62]:
clin['target5'] = clin['ICUvsother'].map({'INTENSIVE CARE UNIT':1, np.nan:0})

In [63]:
sorted(clin.columns)

['AdmitLocation',
 'AgeInYrs',
 'AgeNumber',
 'BP',
 'BloodPressure_LastEDReading',
 'ChartNumber',
 'CleanSubjectiveNotes',
 'DischargeDisposition',
 'DischargeDispositionDesc',
 'Disposition Date & Time',
 'EncounterNumber',
 'Gender',
 'GenderDesc',
 'ICUvsother',
 'ID',
 'InfectionControlScreening',
 'Left ED Date & Time',
 'MainDiagnosisCode',
 'MainDiagnosisCodeDesc',
 'MedicalHistory',
 'O2Saturation_LastEDReading',
 'PIA Date & Time',
 'PatientService',
 'PresentingComplaint',
 'PresentingComplaintDesc',
 'Pulse_LastEDReading',
 'Reg Date & Time',
 'SubjectiveNotes',
 'Temperature_LastEDReading',
 'Triage Date & Time',
 'TriageLevel',
 'diastolic',
 'discharge',
 'dispo',
 'o2sat',
 'outcome',
 'pulse',
 'service',
 'systolic',
 'target',
 'target2',
 'target3',
 'target4',
 'target5',
 'temp']

## splitting into a bunch of sub dataframes and saving them as their own csv files
- loading the excel file is very slow, but loading csv files is faster so I'm making a bunch of dataframes of subsets of data for easy saving/loading/combining

In [64]:
tab_df = clin[['ID', 'TriageLevel',
       'AgeInYrs', 'GenderDesc', 'DischargeDisposition',
       'PresentingComplaint', 'PresentingComplaintDesc', 'AdmitLocation', 'PatientService',
       'BloodPressure_LastEDReading','systolic', 'diastolic','temp','pulse','o2sat']]

In [65]:
target_df = clin[['outcome','target', 'service','target2', 'discharge', 'target3','dispo','target4', 'ICUvsother', 'target5']]

In [66]:
date_df = clin[['Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'Left ED Date & Time']]

## parsing of the medical history into categories

In [67]:
histories = Counter(clin['MedicalHistory']); len(histories)

31331

In [68]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [69]:
for item in history[1000:1050]:
    print (item)

No Significant Medical History
Quad Bypass, Stents x2, <LT>LF>COPD (Chronic Obstructive Pulmonary Disease)<LT>LF>Hypertension (HTN)<LT>LF>High Cholesterol
NIDDM (Non-Insulin-Dependent Diabetes Mellitus)<LT>LF>Asthma<LT>LF>Sleep Apnea
Depression
NIDDM (Non-Insulin-Dependent Diabetes Mellitus)<LT>LF>High BP
Alcoholism
No Significant Medical History
family hx of MI<LT>LF>Thyroid<LT>LF>High Cholesterol
Acid Reflux
Hypertension (HTN)
No Significant Medical History
No Significant Medical History
No Significant Medical History
No Significant Medical History
peripheral neuropathy<LT>LF>NIDDM (Non-Insulin-Dependent Diabetes Mellitus)<LT>LF>fibrotic lung dx, high chol, heart dx, parkinsons
Breast Cancer
Prostate Cancer
No Significant Medical History
No Significant Medical History
High BP<LT>LF>High Cholesterol
Hypertension (HTN)<LT>LF>Thyroid<LT>LF>High Cholesterol
No Significant Medical History
No Significant Medical History
No Significant Medical History
No Significant Medical History
Hyperten

In [70]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace('htn', 'hypertension')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [71]:
replacements = (',','<LT>LF>', '.')
drop_list = ('"', '?')
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
    
    item = item.split('::')
    
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]
    
    item = [fixhx(s) for s in item]
    for d in drop_list:
        item = [x.replace(d,'') for x in item]
    
    item = [x.strip().lower() for x in item if x != 'nan']
    item = [x for x in item if len(x) > 2]
    
    item = list(filter(None, item))
    
    split_hist.append(item)
    

In [72]:
#rejoining them as a string with diagnoses separated by commas
join_hist = [','.join(item) for item in split_hist]

In [73]:
#replacing empty string with NaN
join_hist = [np.nan if item == 'nan' else item for item in join_hist]

In [74]:
len(join_hist)

102543

In [75]:
join_hist[2000:2020]

['no significant medical history',
 'high cholesterol,kidney stones',
 'back pain',
 'benign polyps,cholesterol',
 'no significant medical history',
 'high bp,chf,hypothyroid',
 'prostate cancer,constipation,hypertension',
 'asthma,depression',
 'no significant medical history',
 'pneumonia,bipolar disorder,anxiety',
 'virus in the mouth,healed,autistic',
 'no significant medical history,unable to get medical history',
 'hypertension,high cholesterol,diabetes',
 'no significant medical history',
 'no significant medical history',
 'acid reflux,bypass surgery,pacemaker/cardioverter/defib,high cholesterol,hypertension',
 'anemia',
 'no significant medical history',
 'no significant medical history',
 'no significant medical history']

In [76]:
clin['pmhx'] = join_hist

In [77]:
subj_df = clin[['SubjectiveNotes', 'MedicalHistory', 'pmhx']]

### exploring different diagnoses

In [78]:
diagnoses = Counter()
for hx in split_hist:
    for item in hx:
        diagnoses[item] += 1

In [79]:
#there are 20464 different medical histories used in the dataset!!!
len(diagnoses)

16701

In [80]:
#sorted diagnoses
diagnoses.most_common(40)

[('no significant medical history', 44103),
 ('hypertension', 17032),
 ('high cholesterol', 13371),
 ('niddm', 7171),
 ('high bp', 4756),
 ('depression', 4566),
 ('asthma', 4316),
 ('anxiety', 3493),
 ('thyroid', 3268),
 ('diabetes', 2866),
 ('hypothyroid', 2480),
 ('iddm', 2450),
 ('gerd', 2210),
 ('acid reflux', 1719),
 ('arthritis', 1460),
 ('anemia', 1087),
 ('copd', 1035),
 ('schizophrenia', 838),
 ('kidney stones', 784),
 ('chf', 738),
 ('seizure', 698),
 ('atrial fibrillation', 688),
 ('immunizations up to date', 686),
 ('enlarged prostate', 679),
 ('gout', 655),
 ('cva', 635),
 ('adhd', 612),
 ('migraines', 586),
 ('dementia', 542),
 ('substance misuse', 537),
 ('high chol', 500),
 ('bipolar', 478),
 ('osteoporosis', 466),
 ('chronic back pain', 463),
 ('smoker', 431),
 ('fibromyalgia', 425),
 ('cholesterol', 411),
 ('pacemaker/cardioverter/defib', 394),
 ('pneumonia', 389),
 ('cad', 387)]

In [81]:
#this is a dictionary mapping the rank of the diagnosis to its name
dx_rank_dict = {item[0]:i+1 for i,item in enumerate(diagnoses.most_common())}

#this is a dictionary mapping the frequency of occurence to its name
dx_freq_dict = {item[0]:item[1]/len(diagnoses) for item in diagnoses.most_common()}

In [82]:
#trying to get a feel for how many diagoses are provided for each patient
split_hist_len = [len(item) for item in split_hist]
hxdf = pd.DataFrame(split_hist_len)

In [83]:
hxdf.describe()

,0
count,102543.000000
mean,1.759847
std,1.374310
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,27.000000


In [84]:
lengths = Counter(split_hist_len); lengths

Counter({1: 66562,
         5: 2807,
         6: 1307,
         2: 15334,
         3: 9536,
         8: 323,
         4: 5503,
         9: 167,
         7: 610,
         13: 20,
         0: 187,
         11: 42,
         10: 92,
         12: 27,
         17: 4,
         15: 11,
         14: 5,
         16: 5,
         27: 1})

In [85]:
#most patients have only a single diagnosis, 80% have 2 or less, 96% have 6 or less
for k,v in sorted(lengths.items()):
    print (k, '{0:2f}%'.format(v*100/len(split_hist_len)))

0 0.182363%
1 64.911306%
2 14.953727%
3 9.299513%
4 5.366529%
5 2.737388%
6 1.274587%
7 0.594872%
8 0.314990%
9 0.162859%
10 0.089718%
11 0.040958%
12 0.026330%
13 0.019504%
14 0.004876%
15 0.010727%
16 0.004876%
17 0.003901%
27 0.000975%


In [86]:
#this will create a list for each item where the top 6 diagnoses are listed in order
ordered_hist = []
for item in split_hist:
    #print (item)
    #print (len(item))
    top_6 = []
    for hx in item:
        #print (hx)
        #print (dx_rank_dict[hx])
        #print (dx_rank_dict[hx], hx)
        if item == 'nan':
            top_6.append(np.nan)
        else:
            top_6.append((dx_rank_dict[hx], hx))
    #print (top_6)
    sort = sorted(top_6)
    #print (sort)
    sort2 = [ii for i,ii in sort]
    #print(sort2)
    if len(sort2) > 6:
        sort2 = sort2[:5]
    if len(sort2) < 6:
        pad_req = 6-len(sort2)
        i = 0
        while i < pad_req:
            sort2.append(np.nan) #('n/a')
            i = i+1
        
    #print (top_6)
    #print (sorted(top_6))
    ordered_hist.append(sort2)
    #print ('next item')
    #print ()

In [87]:
medhx_df = pd.DataFrame(ordered_hist, columns = ['medhx' + str(i+1) for i in range(6)])
medhx_df.shape

(102543, 6)

In [88]:
ordered_hist[2000:2020]

[['no significant medical history', nan, nan, nan, nan, nan],
 ['high cholesterol', 'kidney stones', nan, nan, nan, nan],
 ['back pain', nan, nan, nan, nan, nan],
 ['cholesterol', 'benign polyps', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['high bp', 'hypothyroid', 'chf', nan, nan, nan],
 ['hypertension', 'prostate cancer', 'constipation', nan, nan, nan],
 ['depression', 'asthma', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['anxiety', 'pneumonia', 'bipolar disorder', nan, nan, nan],
 ['autistic', 'virus in the mouth', 'healed', nan, nan, nan],
 ['no significant medical history',
  'unable to get medical history',
  nan,
  nan,
  nan,
  nan],
 ['hypertension', 'high cholesterol', 'diabetes', nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['hypertension',
  'high cholesterol',
  'acid reflux',
  'pacemaker/cardiov

In [89]:
comorbids = []

for item in ordered_hist:
    res = [x for x in item if str(x) != 'nan']
    if len(res) == 0: comorbids.append(np.nan)
    else:
        res = [x for x in res if x != 'no significant medical history']
        #print (res, len(res))
        comorbids.append(len(res))

In [90]:
#so now I want to add this to the tabular data frame
tab_df['num_comorbids'] = comorbids

/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [91]:
clin['num_comorbids'] = comorbids

### now I'm going to make a couple of balanced datasets

In [92]:
Counter(clin.target), Counter(clin.target3)

(Counter({1: 92747, 3: 1490, 2: 8048, 4: 258}), Counter({1: 92747, 0: 9796}))

In [93]:
admit_df = clin[clin.target3 == 0]

dc_df = clin[clin.target3==1]
dc_df = dc_df.sample(n= len(admit_df))  #this is the number of admissions from target 3



In [94]:
balanced_df1 = pd.concat([admit_df, dc_df], axis = 0)

In [95]:
icu_df = clin[clin.target2 == 4]
madmit_df = clin[clin.target2 ==3].sample(n = len(icu_df))
sadmit_df = clin[clin.target2 ==2].sample(n = len(icu_df))
disc_df = clin[clin.target2 == 1].sample(n = len(icu_df))

In [96]:
#this is a dataframe with equal numbers of all 4 classes
balanced_df2 = pd.concat([icu_df, madmit_df, sadmit_df, disc_df], axis = 0)

In [97]:
non_icu_df = clin[clin.target2 !=4].sample(n = len(icu_df))

In [98]:
balanced_df3 = pd.concat([icu_df, non_icu_df], axis = 0)

In [99]:
balanced_df3.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'CleanSubjectiveNotes', 'BP', 'systolic', 'diastolic', 'Gender',
       'outcome', 'target', 'service', 'target2', 'discharge', 'target3',
       'dispo', 'target4', 'ICUvsother', 'target5', 'pmhx', 'num_comorbids'],
      dtype='object')

## saving dataframes to csv files

In [100]:
balanced_df1.to_csv(data_path + '/balanced_admit_dc_nlp_data.csv')

balanced_df2.to_csv(data_path +'/balanced_4cls_nlp_data.csv')

balanced_df3.to_csv(data_path +'/balanced_icuvsother_nlp_data.csv')

target_df.to_csv(data_path  + '/targets.csv')

screening_df.to_csv(data_path + '/inf_control_data.csv')

tab_df.to_csv(data_path + '/tabular_data.csv')

date_df.to_csv(data_path + '/data_data.csv')

subj_df.to_csv(data_path + '/subj_data.csv')

medhx_df.to_csv(data_path + '/med_hx.csv')